# Component evaluation

Here we take a CNM object and evaluate all of its components to filter out false positives and only keep real neurons.

The evaluation is done following these criteria:

 - Spatial footprint consistency (rval): The spatial footprint of the component is compared with the frames where this component is active. Other component’s signals are subtracted from these frames, and the resulting raw data is correlated against the spatial component. This ensures that the raw data at the spatial footprint aligns with the extracted trace.
 - Trace signal-noise-ratio (SNR): Peak SNR is calculated from strong calcium transients and the noise estimate.
 - CNN-based classifier (cnn): The shape of components is evaluated by a 4-layered convolutional neural network trained on a manually annotated dataset. The CNN assigns a value of 0-1 to each component depending on its resemblance to a neuronal soma.

Each parameter has a low threshold (*rval_lowest* (default -1), *SNR_lowest* (default 0.5), *cnn_lowest* (default 0.1)) and high threshold (*rval_thr* (default 0.8), *min_SNR* (default 2.5), *min_cnn_thr* (default 0.9)). A component has to exceed ALL low thresholds as well as ONE high threshold to be accepted.

You will have to run the evaluation several times, check how the evaluation looks (if neurons are accepted and noise is rejected), adjust the parameters and run the evaluation again. Repeat this process until (almost) all neurons are accepted with as little false-positives as possible.

In [1]:
# Import packages
import sys
sys.path.append('../custom scripts/')

from caiman.source_extraction import cnmf
import standard_pipeline.place_cell_pipeline as pipe
import standard_pipeline.behavior_import as behavior
import place_cell_class as pc
import caiman as cm
import matplotlib.pyplot as plt
import os
import standard_pipeline.performance_check as performance
import numpy as np
import seaborn as sns

# Start cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)

C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

## Load data

First, we load the CNM object that holds the pre-selected components. We also have to load the memmapped movie, which is used during evaluation.

In [2]:
# Set directory of the session
root = r'W:\Neurophysiology-Storage1\Wahl\Jithin\imaging\M12_Pre_Stroke_Frontal\25\Frontal'

# Load CNM object with pre-selected components
cnm = pipe.load_cnmf(root, 'cnm_pre_selection.hdf5')

# Load movie
mmap_file, images = pipe.load_mmap(root)

Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\imaging\M12_Pre_Stroke_Frontal\25\Frontal\cnm_pre_selection.hdf5...
Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\imaging\M12_Pre_Stroke_Frontal\25\Frontal\memmap__d1_512_d2_472_d3_1_order_C_frames_30000_.mmap...


## Set parameters and run evaluation

Now we set the parameters and run the evaluation. Afterwards, we plot the contours of accepted and rejected components to be able to check the evaluation. Are all neurons accepted? Are obvious non-neurons rejected? If necessary, adjust parameters and run the cell again until the results are satisfactorily (all neurons accepted, as few false positives as possible).

In [3]:
# Component evaluation

# plot graphs as separate, interactive window
%matplotlib qt     

min_SNR = 8  # signal to noise ratio for accepting a component (default 2)
SNR_lowest = 3
rval_thr = 0.95  # space correlation threshold for accepting a component (default 0.85)
rval_lowest = -1
cnn_thr = 0.95  # threshold for CNN based classifier (default 0.99)
cnn_lowest = 0.3  # neurons with cnn probability lower than this value are rejected (default 0.1)

cnm.params.set('quality', {'SNR_lowest': SNR_lowest,
                           'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'rval_lowest': rval_lowest,
                           'use_cnn': True,
                           'min_cnn_thr': cnn_thr,
                           'cnn_lowest': cnn_lowest})
cnm = pipe.run_evaluation(images, cnm, dview=dview)

cnm.estimates.plot_contours(img=cnm.estimates.Cn, idx=cnm.estimates.idx_components, display_numbers=True)

USING MODEL:C:\Users\Jnambi\caiman_data\model\cnn_model.json
406/406 [==============================] - 0s 844us/sample


In [5]:
cnm.estimates.plot_contours(img=cnm.estimates.Cn, idx=cnm.estimates.idx_components, display_numbers=True)

### Check single component

To be able to better tune evaluation parameters, its possible to check the evaluation results of single components. This is possible with the function ``pipe.check_eval_results()``.

This function needs the CNM object as a first argument and the ID of the requested cell. The ID is from the cnm.estimates.idx_components (if you want to check accepted components) or cnm.estimates.idx_components_bad (if you want to check rejected components). The index is the number displayed next to/inside the contour minus 1 (contours start counting at 1, python at 0).

In [ ]:
pipe.check_eval_results(cnm, cnm.estimates.idx_components[113], plot_contours=True)

In [ ]:
# Plot trace to check for real cells/transients
plt.figure()
plt.plot(cnm.estimates.C[20])

In [ ]:
# Manually accept cells if necessary (use indices of idx_components_bad)
cnm = pipe.accept_cells(cnm, [365])

In [4]:
# Manually reject cells if necessary (use indices of idx_components)
cnm = pipe.reject_cells(cnm, [1,4,24,69,145,168])

## Save results

If the evaluation is good enough, the CaImAn pipeline is coming to an end. Select the components (which removes the data of all rejected components, so be sure about this step), detrend the calcium activity (dF/F), export the results as a contour plot and finally save the complete CNM object.

In [6]:
# Save the CNM object once before before deleting the data of rejected components
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_pre_selection.hdf5'), verbose=False, overwrite=True)

# Select components, which keeps the data of accepted components and deletes the data of rejected ones
cnm.estimates.select_components(use_object=True)

# Detrend calcium data (compute dF/F). Note that the frame window should be big enough to be larger than the longest
# actual transients, but shorter than slow changes in background fluorescence changes (e.g. due to slow z-drift)
# A good measure is to adapt the window size to the movie length, e.g. by taking a fraction of the duration
# Note that a larger frame window also increases the time of dF/F calculation
#cnm.params.data['dff_window'] = int(len(cnm.estimates.C[0])/5)
cnm.params.data['dff_window'] = 3000
cnm.estimates.detrend_df_f(quantileMin=8, frames_window=cnm.params.data['dff_window'])

# Save complete CNMF results
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_results.hdf5'), overwrite=False, verbose=False)

# Plot contours of all accepted components
cnm.estimates.plot_contours(img=cnm.estimates.Cn, display_numbers=False)
plt.tight_layout()
fig = plt.gcf()
fig.set_size_inches((10, 10))
plt.savefig(os.path.join(root, 'components.png'))
plt.close()

All done!

Now you can proceed with the actual data analysis and [place cell detection](.\\Place_Cell_Detection.ipynb).